# I saved pre processed text in csv file m using that dataset

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk import WordNetLemmatizer
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.optimizers import Adam

In [3]:
urdu_dataset = pd.read_csv("Analysed_Urdu_Tweets.csv",index_col=False)
urdu_dataset.drop(urdu_dataset.columns[0],axis=1,inplace=True)
urdu_dataset.head()


,Tweet,Class
0,ایٹم بم بنایا ھے بھائی ایٹم بمب کوٹ لکھپت اتفا...,2
1,چندے انقلاب عمران خان وزیر اعظم بن سکتے,0
2,ٹویٹر خیال,1
3,سرچ انجن گوگل نائب صدر فضا فٹ بلندی چھلانگ عال...,2
4,اسکی لہریں یار أ,2


In [4]:

## WORDS in dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(urdu_dataset['Tweet'])
sequences = tokenizer.texts_to_sequences(urdu_dataset['Tweet'])

max_sequence_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
## Split the dataset
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, urdu_dataset['Class'], test_size=0.25, random_state=42)

print("Training set shape:",X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (749, 31)
Testing set shape: (250, 31)


# Models

In [9]:
import gensim.downloader as api
import tensorflow_hub as hub
word2vec_model = api.load("word2vec-google-news-300")
# glove_model = api.load("glove-twitter-200")
# fasttext_model = api.load("fasttext-wiki-news-subwords-300")
# elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [10]:
max_length = max([len(seq) for seq in X_train])
X_train_padded = pad_sequences(X_train, maxlen=max_length)
X_test_padded = pad_sequences(X_test, maxlen=max_length)

In [11]:
# word2vec_weights = create_embedding_matrix(word2vec_model, tokenizer)

vocab_size = len(tokenizer.word_index) + 1
embed_matrix = np.zeros((vocab_size,word2vec_model.vector_size))

for w,i in tokenizer.word_index.items():
    if w in word2vec_model:
        embed_matrix[i] = word2vec_model[w]
word2vec_weights = embed_matrix

In [12]:
# GRU -> 3,0.7 ->0.66
# nested for for dropout and _
num_layers =3
dropout_rate=0.7

embedding_dim = 128

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(GRU(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(GRU(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.25)

Epoch 1/8
18/18 [==============================] - 25s 268ms/step - loss: 0.0000e+00 - accuracy: 0.2977 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 2/8
18/18 [==============================] - 2s 86ms/step - loss: 0.0000e+00 - accuracy: 0.3422 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 3/8
18/18 [==============================] - 1s 82ms/step - loss: 0.0000e+00 - accuracy: 0.3672 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 4/8
18/18 [==============================] - 1s 81ms/step - loss: 0.0000e+00 - accuracy: 0.4492 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 5/8
18/18 [==============================] - 1s 77ms/step - loss: 0.0000e+00 - accuracy: 0.4795 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 6/8
18/18 [==============================] - 1s 76ms/step - loss: 0.0000e+00 - accuracy: 0.4938 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 7/8
18/18 [==============================] - 1s 75ms/step - loss: 0.0000e+00 - accuracy: 0.4848 - val_lo

In [16]:
model_eval = dict()
from sklearn.metrics import precision_score, recall_score, f1_score

In [19]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
model_eval['GRU(no embedding)'] = {'Precision':precision,'Recall':recall,'F1 score':f1}

8/8 [==============================] - 0s 20ms/step


In [20]:
model_eval

{'GRU(no embedding)': {'Precision': 0.16266666666666665,
  'Recall': 0.3333333333333333,
  'F1 score': 0.2186379928315412}}

In [24]:
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_weights), output_dim=300,weights=[word2vec_weights], input_length=max_length))
model.add(GRU(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(GRU(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
10/10 [==============================] - 33s 380ms/step - loss: 0.0000e+00 - accuracy: 0.1753 - val_loss: 0.0000e+00 - val_accuracy: 0.0067
Epoch 2/10
10/10 [==============================] - 1s 140ms/step - loss: 0.0000e+00 - accuracy: 0.3222 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 3/10
10/10 [==============================] - 2s 157ms/step - loss: 0.0000e+00 - accuracy: 0.3105 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 4/10
10/10 [==============================] - 2s 154ms/step - loss: 0.0000e+00 - accuracy: 0.4324 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 5/10
10/10 [==============================] - 2s 152ms/step - loss: 0.0000e+00 - accuracy: 0.4908 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 6/10
10/10 [==============================] - 2s 157ms/step - loss: 0.0000e+00 - accuracy: 0.4958 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 7/10
10/10 [==============================] - 1s 139ms/step - loss: 0.0000e+00 - accuracy: 0.

In [30]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision2 = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall2 = recall_score(y_test, y_pred, average='macro', zero_division=0)
f12 = f1_score(y_test, y_pred, average='macro', zero_division=0)

8/8 [==============================] - 1s 25ms/step


In [31]:
model_eval['GRU(embedding: word2vec)'] = {'Precision':precision2,'Recall':recall2,'F1 score':f12}

In [32]:
table = pd.DataFrame.from_dict(model_eval, orient='index')
print(table)

                          Precision    Recall  F1 score
GRU(no embedding)          0.162667  0.333333  0.218638
GRU(embedding: word2vec)   0.162667  0.333333  0.218638
